# 农场规划

## 目标和前提条件

通过这个示例培养您的建模技能，您将学习如何解决一个复杂的多周期生产规划问题，该问题涉及优化一个农场在五年内的运营。

关于这类模型的更多信息可以在H. Paul Williams所著的《数学规划中的模型构建》第五版的示例#8中找到，具体在第262-263页和第312-315页。

这是一个高级建模示例，我们假设您已经掌握Python和Gurobi Python API，并且具有构建数学优化模型的进阶知识。通常，这些示例的目标函数和/或约束条件比较复杂，或需要使用Gurobi Python API的高级特性。

**下载代码库** <br /> 您可以通过点击[这里](https://github.com/Gurobi/modeling-examples/archive/master.zip)下载包含本示例和其他示例的代码库。

---
## 问题描述

一位拥有200英亩农场的农民希望制定一个五年生产计划。

目前，他拥有一个由120头奶牛组成的牛群，其中包括20头小母牛（年轻母牛）和100头成年奶牛。每头小母牛需要2/3英亩的土地来维持，每头奶牛需要一英亩。

平均而言，一头奶牛每年生产1.1头小牛。其中一半的小牛将是公牛犊，出生后不久就会以平均每头$\$30$的价格出售。剩余的小母牛可以以$\$40$的价格出售，或者饲养到两岁时成为奶牛。对于当年，所有准备出售的小母牛已经售出。

一般做法是在12岁时将所有奶牛以平均每头$\$120$的价格出售。然而，每年平均有$5\%$的小母牛和$2\%$的奶牛会死亡。目前，农民的120头牛群均匀分布，从新生到11岁每个年龄段都有10头牛。

奶牛的牛奶每年可以卖到$\$370$。农民目前最多可以容纳130头牛，但这个容量限制可以通过每增加一头牛支付$\$200$来扩大。

每头奶牛每年需要0.6吨谷物和0.7吨甜菜。这两种作物都可以在农场种植。每英亩可以产出1.5吨甜菜。然而，只有80英亩的土地适合种植谷物，这些土地的生产力水平如下：

| 土地组 | 面积（英亩） | 谷物产量（吨/英亩） |
| --- | --- | --- |
| 第1组 | 20 | 1.10 |
| 第2组 | 30 | 0.90 |
| 第3组 | 20 | 0.80 |
| 第4组 | 10 | 0.65 |

甜菜可以以每吨$\$70$的价格购买，以每吨$\$58$的价格出售。谷物可以以每吨$\$90$的价格购买，以每吨$\$75$的价格出售。

牛的饲养以及谷物和甜菜生产的年度劳动力需求如下：

| <i></i> | 所需劳动力（小时/年） |
| --- | --- |
| 小母牛 | 10 |
| 产奶奶牛 | 42 |
| 种植谷物的每英亩土地 | 4 |
| 种植甜菜的每英亩土地 | 14 |

其他年度成本如下：

| <i></i> | 成本（美元/年） |
| --- | --- |
| 小母牛 | 50 |
| 产奶奶牛 | 100 |
| 种植谷物的每英亩土地 | 15 |
| 种植甜菜的每英亩土地 | 10 |

劳动力目前每年花费农民$\$4000$，这笔费用提供5,500小时的劳动力。额外的劳动力可以按每小时$\$1.20$的价格支付。

任何资本支出都可以通过10年期贷款以15%的年利率融资。利息和本金以10年等额年金的方式偿还。禁止年度现金流为负。

农民不希望在五年期结束时奶牛总数减少超过50%，也不希望增加超过75%。

农民在未来五年应该遵循什么计划以最大化利润？

---
## 模型公式化

### 集合和索引

$t \in \text{Years}=\{1,2,\dots,5\}$：年份集合。

$l \in \text{Lands}=\{1,2,3,4\}$：土地组集合。

$k \in \text{Ages}=\{1,2,\dots,12\}$：牛的年龄集合。

### 参数

$\text{Installment} \in \mathbb{R}^+$：每$\$200$贷款的年度支付额。可以按如下方式计算：

$$
\text{Installment}= \text{Loan}*r*\frac{(1+r)^n}{(1+r)^n-1},
$$

其中$\text{Loan}$是$\$200$的贷款金额，$r$是15%的利率，$n$代表十年期限。注意，书中报告的值是$\$39.71$而不是$\$39.85$。

$\text{Housing_cap} \in \mathbb{N}$：可以容纳的牛的数量。

$\text{Land_cap} \in \mathbb{R}^+$：可用土地（英亩）。

$\text{Labor_cap} \in \mathbb{R}^+$：一年中可用的常规劳动力（小时）。

$\text{GR_intake} \in \mathbb{R}^+$：一头奶牛一年消耗的谷物吨数。

$\text{SB_intake} \in \mathbb{R}^+$：一头奶牛一年消耗的甜菜吨数。

$\text{HF_land} \in \mathbb{R}^+$：维持每头小母牛所需的英亩数。

$\text{HF_labor} \in \mathbb{R}^+$：每头小母牛一年所需的劳动力小时数。

$\text{Cow_labor} \in \mathbb{R}^+$：每头奶牛一年所需的劳动力小时数。

$\text{GR_labor} \in \mathbb{R}^+$：每英亩种植谷物的土地一年所需的劳动力小时数。

$\text{SB_labor} \in \mathbb{R}^+$：每英亩种植甜菜的土地一年所需的劳动力小时数。

$\text{HF_decay} \in [0,1] \subset \mathbb{R}^+$：小母牛每年平均死亡的比例。

$\text{Cow_decay} \in [0,1] \subset \mathbb{R}^+$：奶牛每年平均死亡的比例。

$\text{Birthrate} \in \mathbb{R}^+$：一头奶牛每年预期生产的小牛数。

$\text{Min_final_cows} \in \mathbb{N}$：规划期结束时的最少奶牛数。

$\text{Max_final_cows} \in \mathbb{N}$：规划期结束时的最多奶牛数。

$\text{Initial_HF} \in \mathbb{R}^+$：规划期开始时每个年龄的小母牛数量。

$\text{Initial_cows} \in \mathbb{R}^+$：规划期开始时每个年龄的奶牛数量。

$\text{BL_price} \in \mathbb{R}^+$：出售一头公牛犊的价格。

$\text{HF_price} \in \mathbb{R}^+$：出售一头小母牛的价格。

$\text{Cow_price} \in \mathbb{R}^+$：出售一头奶牛的价格。

$\text{Milk_price} \in \mathbb{R}^+$：一头奶牛一年生产的牛奶的销售价格。

$\text{GR_price} \in \mathbb{R}^+$：出售一吨谷物的价格。

$\text{SB_price} \in \mathbb{R}^+$：出售一吨甜菜的价格。

$\text{GR_cost} \in \mathbb{R}^+$：购买一吨谷物的成本。

$\text{SB_cost} \in \mathbb{R}^+$：购买一吨甜菜的成本。

$\text{Overtime_cost} \in \mathbb{R}^+$：获取一小时加班的成本。

$\text{Regular_time_cost} \in \mathbb{R}^+$：获取5,500小时常规劳动力的成本。

$\text{HF_cost} \in \mathbb{R}^+$：维持一头小母牛的年度成本。

$\text{Cow_cost} \in \mathbb{R}^+$：维持一头奶牛的年度成本。

$\text{GR_land_cost} \in \mathbb{R}^+$：维持一英亩种植谷物土地的年度成本。

$\text{SB_land_cost} \in \mathbb{R}^+$：维持一英亩种植甜菜土地的年度成本。

$\text{SB_yield} \in \mathbb{R}^+$：甜菜产量。

$\text{GR_yield}_l \in \mathbb{R}^+$：土地组$l$的谷物产量。

$\text{GR_area}_l \in \mathbb{R}^+$：土地组$l$中适合种植谷物的英亩数。

### 决策变量

$\text{Outlay}_t \in \mathbb{R}^+$：第$t$年用于租用牛舍的金额（以$\$200$为单位）。

$\text{Overtime}_t \in \mathbb{R}^+$：第$t$年需要的额外劳动力小时数。

$\text{Newborn}_t \in \mathbb{R}^+$：第$t$年留下饲养的新生小母牛数量。

$\text{HF_sell}_t \in \mathbb{R}^+$：第$t$年要出售的新生小母牛数量。

$\text{Profit}_t \in \mathbb{R}^+$：第$t$年获得的利润。

$\text{SB_buy}_t \in \mathbb{R}^+$：第$t$年要购买的甜菜吨数。

$\text{SB_sell}_t \in \mathbb{R}^+$：第$t$年要出售的甜菜吨数。

$\text{GR_buy}_t \in \mathbb{R}^+$：第$t$年要购买的谷物吨数。

$\text{GR_sell}_t \in \mathbb{R}^+$：第$t$年要出售的谷物吨数。

$\text{SB}_t \in \mathbb{R}^+$：第$t$年要种植的甜菜吨数。

$\text{GR}_{t,l} \in \mathbb{R}^+$：第$t$年在土地组$l$上要种植的谷物吨数。

$\text{Cows}_{t,k} \in \mathbb{R}^+$：第$t$年年龄为$k$的可用牛的数量。

### 目标函数

- **利润**：最大化规划期内的总利润（美元）。注意，为了使后期的资本支出与前期一样昂贵，有必要减去未偿还的贷款支付。

\begin{equation}
\text{最大化} \quad Z = \sum_{t \in \text{Years}}{\text{Profit}_t - installment*(t+4)*\text{Outlay}_t}
\end{equation}

### 约束条件

- **牛舍容量**：第$t$年的牲畜数量不能超过已安装容量加上牛舍租用量。

\begin{equation}
\text{Newborn}_t + \sum_{k \in \text{Ages} \setminus \{12\}}{\text{Cows}_{t,k}} \leq \text{Housing_cap} + \sum_{d \in \text{Years}: d \leq t}{\text{Outlay}_d} \quad \forall t \in \text{Years}
\end{equation}

- **食物消耗**：第$t$年必须有足够的食物来饲养牲畜。

- 谷物。

\begin{equation}
\sum_{k \in \text{Ages} \setminus \{1,12\}}{\text{GR_intake}*\text{Cows}_{t,k}} \leq \text{GR_buy}_t - \text{GR_sell}_t + \sum_{l \in \text{Lands}}{\text{GR}_{t,l}} \quad \forall t \in \text{Years}
\end{equation}

- 甜菜。

\begin{equation}
\sum_{k \in \text{Ages} \setminus \{1,12\}}{\text{SB_intake}*\text{Cows}_{t,k}} \leq \text{SB_buy}_t - \text{SB_sell}_t + \text{SB}_t \quad \forall t \in \text{Years}
\end{equation}

- **谷物种植**：土地$l$上生产的谷物不能超过第$t$年的生产能力。

\begin{equation}
\text{GR}_{t,l} \leq \text{GR_yield}_l*\text{GR_area}_l \quad \forall (t,l) \in \text{Years} \times \text{Lands}
\end{equation}

- **土地容量**：第$t$年的空间使用不能超过可用土地。

\begin{equation}
\frac{1}{\text{SB_yield}}*\text{SB}_t + \text{HF_land}*(\text{Newborn}_t + \text{Cow}_{t,1}) + 
\end{equation}

\begin{equation}
\sum_{k \in \text{Ages} \setminus \{1,12\}}{\text{Cows}_{t,k}} + 
\end{equation}

\begin{equation}
\sum_{l \in \text{Lands}}{\frac{\text{GR}_{t,l}}{\text{GR_yield}_l}} \leq \text{Land_cap} \quad \forall t \in \text{Years}
\end{equation}

- **劳动力**：第$t$年经营农场所需的劳动力不能超过合同时间加上加班时间。

\begin{equation}
\text{HF_labor}*(\text{Newborn}_t + \text{Cow}_{t,1}) + 
\end{equation}

\begin{equation}
\sum_{k \in \text{Ages} \setminus \{1,12\}}{\text{Cow_labor}*\text{Cow}_{t,k}} + 
\end{equation}

\begin{equation}
\sum_{l \in \text{Lands}}{\frac{\text{GR_labor}*\text{GR}_{t,l}}{\text{GR_yield}_l}} + 
\end{equation}

\begin{equation}
\frac{\text{SB_labor}*\text{SB}_t}{\text{SB_yield}} \leq \text{Labor_cap} + \text{Overtime}_t \quad \forall t \in \text{Years}
\end{equation}

- **连续性**：第$t$年的牲畜必须从上一年存活下来。

\begin{equation}
\text{Cows}_{t,1} = (1-\text{HF_decay})*\text{Newborn}_{t-1} \quad \forall t \in \text{Years} \setminus \{1\}
\end{equation}

\begin{equation}
\text{Cows}_{t,2} = (1-\text{HF_decay})*\text{Cows}_{t-1,1} \quad \forall t \in \text{Years} \setminus \{1\}
\end{equation}

\begin{equation}
\text{Cows}_{t,k+1} = (1-\text{Cow_decay})*\text{Cows}_{t-1,k} \quad \forall (t,k) \in \text{Years} \setminus \{1\} \times \text{Ages} \setminus \{1,12\}
\end{equation}

- **小母牛出生**：第$t$年出生的小母牛数量取决于奶牛的数量。

\begin{equation}
\text{Newborn}_t + \text{HF_sell}_t = \sum_{k \in \text{Ages} \setminus \{1,12\}}{\frac{\text{Birthrate}}{2}*\text{Cows}_{t,k}} \quad \forall t \in \text{Years}
\end{equation}

- **最终奶牛数量**：规划期结束时的奶牛数量必须在容忍范围内。

\begin{equation}
\text{Min_final_cows} \leq \sum_{k \in \text{Ages} \setminus \{1,12\}}{\text{Cows}_{5,k}} \leq \text{Max_final_cows}
\end{equation}

- **初始条件**：设置规划期开始时可用的牲畜数量。

\begin{equation}
\text{Cows}_{1,1} = \text{Initial_HF}
\end{equation}

\begin{equation}
\text{Cows}_{1,2} = \text{Initial_HF}
\end{equation}

\begin{equation}
\text{Cows}_{1,k} = \text{Initial_cows} \quad \forall k \in \text{Ages} \setminus \{1,2\}
\end{equation}

- **年度利润**：第$t$年的利润由作物和牲畜的运营驱动，在考虑劳动力、土地和财务成本后计算。

\begin{equation}
\text{Profit}_t = \frac{\text{BL_price}*\text{Birthrate}}{2}*\sum_{k \in \text{Ages} \setminus \{1,12\}}{\text{Cows}_{t,k}}
+ \text{HF_price}*\text{HF_sell}_t
+ \text{Cow_price}*\text{Cows}_{t,12}
\end{equation}

\begin{equation}
+ \text{Milk_price}*\sum_{k \in \text{Ages} \setminus \{1,12\}}{\text{Cows}_{t,k}}
\end{equation}

\begin{equation}
+ \text{GR_price}*\text{GR_sell}_t
+ \text{SB_price}*\text{SB_sell}_t
\end{equation}

\begin{equation}
- \text{GR_cost}*\text{GR_buy}_t
- \text{SB_cost}*\text{SB_buy}_t
\end{equation}

\begin{equation}
- \text{Overtime_cost}*\text{Overtime}_t
- \text{Regular_time_cost}
\end{equation}

\begin{equation}
- \text{HF_cost}*(\text{Newborn}_t + \text{Cows}_{t,1})
- \text{Cow_cost}*\sum_{k \in \text{Ages} \setminus \{1,12\}}{\text{Cows}_{t,k}}
\end{equation}

\begin{equation}
- \text{GR_land_cost}*\sum_{l \in \text{Lands}}{\frac{\text{GR}_{t,l}}{\text{GR_yield}_l}}
- \text{SB_land_cost}*\frac{\text{SB}_t}{\text{SB_yield}} \\
- \text{Installment}*\sum_{d \in \text{Years}:d \leq t}{\text{Outlay}_d}
\quad \forall t \in \text{Years}
\end{equation}

---
## Python实现

我们导入Gurobi Python模块和其他Python库。

In [ ]:
%pip install gurobipy

In [ ]:
import numpy as np
import pandas as pd

import gurobipy as gp
from gurobipy import GRB

# 使用Python 3.11和Gurobi 11.0进行测试

## 输入数据
我们定义模型的所有输入数据。

In [ ]:
# 参数

years = [1,2,3,4,5]
lands = [1,2,3,4]
ages = [1,2,3,4,5,6,7,8,9,10,11,12]
cow_ages = [2,3,4,5,6,7,8,9,10,11]

gr_area = {1: 20.0, 2: 30.0, 3: 20.0, 4: 10.0}
gr_yield = {1: 1.1,  2: 0.9, 3: 0.8, 4:0.65}
sb_yield = 1.5
housing_cap = 130
gr_intake = 0.6
sb_intake = 0.7
hf_land = 2/3.0
land_cap = 200
hf_labor = 10/100.0
cow_labor = 42/100.0
gr_labor = 4/100.0
sb_labor = 14/100.0
labor_cap = 5500/100.0
cow_decay = 0.02
hf_decay = 0.05
initial_hf = 9.5
initial_cows = 9.8
birthrate = 1.1
min_final_cows = 50
max_final_cows = 175
bl_price = 30
hf_price = 40
cow_price = 120
milk_price = 370
gr_price = 75
sb_price = 58
gr_cost = 90
sb_cost = 70
overtime_cost = 120
regular_time_cost = 4000
hf_cost = 50
cow_cost = 100
gr_land_cost = 15
sb_land_cost = 10
installment = 39.71

## 模型部署

我们创建一个模型和变量。每年都有连续变量,用来表示:种植的甜菜数量(以吨为单位)、购买的谷物量(以吨为单位)、出售的谷物量(以吨为单位)、购买的甜菜量(以吨为单位)、出售的甜菜量(以吨为单位)、额外招募的劳动力、资本支出额、出生时出售的小母牛数量、利润以及新生牛(0岁)的数量。

对于每一年和每个土地组,都有一个连续变量,用来表示在该土地组上种植的谷物数量。对于每一年和每个牛的年龄,都有一个连续变量,用来表示当前年份中该年龄的牛的数量。

In [3]:
model = gp.Model('Farming')
sb = model.addVars(years, vtype=GRB.CONTINUOUS, name="SB")
gr_buy = model.addVars(years, vtype=GRB.CONTINUOUS, name="GR_buy")
gr_sell = model.addVars(years, vtype=GRB.CONTINUOUS, name="GR_sell")
sb_buy = model.addVars(years, vtype=GRB.CONTINUOUS, name="SB_buy")
sb_sell = model.addVars(years, vtype=GRB.CONTINUOUS, name="SB_sell")
overtime = model.addVars(years, vtype=GRB.CONTINUOUS, name="Overtime")
outlay = model.addVars(years, vtype=GRB.CONTINUOUS, name="Outlay")
hf_sell = model.addVars(years, vtype=GRB.CONTINUOUS, name="HF_sell")
newborn = model.addVars(years, vtype=GRB.CONTINUOUS, name="Newborn")
profit = model.addVars(years, vtype=GRB.CONTINUOUS, name="Profit")
gr = model.addVars(years, lands, vtype=GRB.CONTINUOUS, name="GR")
cows = model.addVars(years, ages, vtype=GRB.CONTINUOUS, name="Cows")

Using license file c:\gurobi\gurobi.lic


接下来我们插入约束条件:

每年牛舍的容量只能容纳130头牛。实际上可能会有超过130头牛,但是要为这些额外的牛提供住所需要额外的租赁费用,这部分费用由`Outlay`变量来表示。

In [ ]:
# 1. 牛舍容量

HousingCap = model.addConstrs((newborn[year] +
                    cows[year,1] +
                    gp.quicksum(cows[year,age] for age in cow_ages) -
                    gp.quicksum(outlay[d] for d in years if d <= year)
                    <= housing_cap for year in years), name="Housing_cap")

售出、购买和生产谷物和甜菜后,储存中必须有足够的数量来饲养所有牛。

In [ ]:
# 2.1 食物消耗 (谷物)

GrainConsumption = model.addConstrs((gp.quicksum(gr_intake*cows[year, age] for age in cow_ages)
                  <= gr_buy[year] - gr_sell[year] + gr.sum(year, '*')
                  for year in years), name="Grain_consumption")

# 2.1 食物消耗 (甜菜)
SugarbeetConsumption = model.addConstrs((gp.quicksum(sb_intake*cows[year, age] for age in cow_ages)
                  <= sb_buy[year] - sb_sell[year] + sb[year]
                  for year in years), name="Sugar_beet_consumption")

在一个土地组上生产的谷物不能超过该土地组指定的生产能力。

In [ ]:
# 3. 谷物种植

GrainGrowing = model.addConstrs((gr[year, land] <= gr_yield[land]*gr_area[land]
                  for year in years for land in lands), name="Grain_growing")

每头牛需要一定数量的土地来维持。所需数量取决于牛的年龄。最多有200英亩的可用土地。

In [ ]:
# 4. 土地容量

LandCap = model.addConstrs((sb[year]/sb_yield + hf_land*(newborn[year] + cows[year,1])
                  + gp.quicksum((1/gr_yield[land])*gr[year, land] for land in lands)
                  + gp.quicksum(cows[year, age] for age in cow_ages)
                  <= land_cap for year in years), name="Land_capacity")

每头牛和每英亩土地都需要一定的工作时间来维护。农场目前每年能够提供固定数量的工作小时。任何需要额外完成的工作都需要外部工人,这将产生额外的成本。

In [ ]:
# 5. 劳动力

Labor = model.addConstrs((hf_labor*(newborn[year] + cows[year,1])
                  + gp.quicksum(cow_labor*cows[year, age] for age in cow_ages)
                  + gp.quicksum(gr_labor/gr_yield[land]*gr[year,land] for land in lands)
                  + sb_labor/sb_yield*sb[year] 
                  <= labor_cap + overtime[year] for year in years), name="Labor")

每年都有一定比例的牛会死亡,这取决于它们的年龄。

In [ ]:
# 6.1 连续性

Continuity1 = model.addConstrs((cows[year,1] == (1-hf_decay)*newborn[year-1] 
                  for year in years if year > min(years)),
                 name="Continuity_a")

# 6.2 连续性

Continuity2 = model.addConstrs((cows[year,2] == (1-hf_decay)*cows[year-1,1] 
                  for year in years if year > min(years)),
                 name="Continuity_b")

# 6.3 连续性

Continuity3 = model.addConstrs((cows[year,age+1] == (1-cow_decay)*cows[year-1,age] 
                  for year in years for age in cow_ages if year > min(years)),
                 name="Continuity_c")

跟踪牛的数量;牛可以通过购买、出售或出生进入/退出模型。

In [ ]:
# 7. 小母牛出生

HeifersBirth = model.addConstrs((newborn[year] + hf_sell[year] 
                  == gp.quicksum(birthrate/2*cows[year,age] for age in cow_ages) for year in years)
                 , name="Heifers_birth")

在五年结束时,农民希望至少有50头、最多有175头奶牛。

In [ ]:
# 8. 最终奶牛数量
FinalDairyCows = model.addRange(gp.quicksum(cows[max(years), age] for age in cow_ages), min_final_cows, max_final_cows, name="Final_dairy_cows" )

在第一年,有9.5头一岁牛和9.5头二岁牛。此外,从三岁到十二岁的每个年龄段都有9.8头牛。注意我们将其作为线性规划模型来求解以使其更容易解决。这可能会导致变量出现小数值,而这些变量在现实中是整数。

In [ ]:
# 9.1-9.2 初始条件

InitialHeifers = model.addConstrs((initial_hf == cows[1, age] for age in ages if age < 3),
                 name="Initial_conditions")

# 9.3 初始条件

InitialCows = model.addConstrs((initial_cows == cows[1, age] for age in ages if age >= 3),
                 name="Initial_condition_cows")

以下约束条件确定年度利润。目前劳动力的总成本是$\$4,000$。利润受以下因素影响:公牛犊和小母牛的出售、12岁牛的出售、牛奶销售、谷物销售、甜菜销售、谷物购买、甜菜购买、劳动力成本、小母牛成本、奶牛成本、谷物成本、甜菜成本和资本成本。

In [ ]:
# 10. 年度利润

YearlyProfit = model.addConstrs((profit[year]
                  == bl_price*birthrate/2*gp.quicksum(cows[year, age] for age in cow_ages)
                  + hf_price*hf_sell[year] + cow_price*cows[year, 12]
                  + milk_price*gp.quicksum(cows[year, age] for age in cow_ages)
                  + gr_price*gr_sell[year] + sb_price*sb_sell[year]
                  - gr_cost*gr_buy[year] - sb_cost*sb_buy[year]
                  - overtime_cost*overtime[year] - regular_time_cost
                  - hf_cost*(newborn[year] + cows[year,1])
                  - cow_cost*gp.quicksum(cows[year, age] for age in cow_ages)
                  - gr_land_cost*gp.quicksum(gr[year, land]/gr_yield[land] for land in lands)
                  - sb_land_cost*sb[year]/sb_yield
                  - installment*gp.quicksum(outlay[d] for d in years if d <= year)
                  for year in years), name="Yearly_profit")

规划期内的总利润包括计算所得的利润减去尚未偿还的贷款支付:

In [ ]:
# 0. 总利润

model.setObjective(gp.quicksum(profit[year] - installment*(year+4)*outlay[year] for year in years), GRB.MAXIMIZE)

接下来,我们开始优化过程,Gurobi找到最优解。  

In [15]:
model.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 116 rows, 131 columns and 734 nonzeros
Model fingerprint: 0x1612a027
Coefficient statistics:
  Matrix range     [4e-02, 3e+02]
  Objective range  [1e+00, 4e+02]
  Bounds range     [1e+02, 1e+02]
  RHS range        [7e+00, 4e+03]
Presolve removed 84 rows and 67 columns
Presolve time: 0.01s
Presolved: 32 rows, 64 columns, 252 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.1200000e+32   5.000000e+30   5.120000e+02      0s
      24    1.2171917e+05   0.000000e+00   0.000000e+00      0s

Solved in 24 iterations and 0.01 seconds
Optimal objective  1.217191729e+05


---
## 分析

在该模型涵盖的五年期间内,最优计划产生的总利润为$\$121,719.17$。以下是每年的详细计划。

### 财务计划

该计划确定了规划期内每个时期的利润和支出。例如,第1年的利润为$\$21,906.1$,支出为零。

In [16]:
rows = ["Profit", "Outlay"]
columns = years.copy()
finance_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)

for year in years:
    if (abs(profit[year].x) > 1e-6):
        finance_plan.loc["Profit", year] = np.round(profit[year].x, 1)
    if (abs(outlay[year].x) > 1e-6):
        finance_plan.loc["Outlay", year] = np.round(outlay[year].x, 1)
finance_plan

,1,2,3,4,5
Profit,21906.1,21888.7,25816.1,26825.8,25282.6
Outlay,0.0,0.0,0.0,0.0,0.0


### 谷物计划
该计划规定了在规划期内每一年(列)每种土地组(行)要种植的谷物吨数。它还规定了在规划期内每一年要购买和出售的谷物吨数。例如,我们将在第2年在1号土地组种植22吨谷物。此外,我们将在第2年购买35.1吨谷物。

In [17]:
rows = lands.copy() + ["Buy", "Sell"]
columns = years.copy()
gr_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)

for year, land in gr.keys():
    if (abs(gr[year, land].x) > 1e-6):
        gr_plan.loc[land, year] = np.round(gr[year, land].x, 1)
for year in years:
    if (abs(gr_buy[year].x) > 1e-6):
        gr_plan.loc["Buy", year] = np.round(gr_buy[year].x, 1)
    if (abs(gr_sell[year].x) > 1e-6):
        gr_plan.loc["Sell", year] = np.round(gr_sell[year].x, 1)
gr_plan

,1,2,3,4,5
1,22.0,22.0,22.0,22.0,22.0
2,0.0,0.0,2.8,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0
Buy,36.6,35.1,37.8,40.1,33.5
Sell,0.0,0.0,0.0,0.0,0.0


### 甜菜计划
该计划规定了在规划期内每一年(列)要种植的甜菜吨数。它还规定了在规划期内每一年要购买和出售的甜菜吨数。例如,我们将在第2年种植94吨甜菜。此外,我们将在第2年出售27.4吨甜菜。

In [18]:
rows = ["Grow", "Buy", "Sell"]
columns = years.copy()
sb_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)

for year in years:
    if (abs(sb[year].x) > 1e-6):
        sb_plan.loc["Grow", year] = np.round(sb[year].x, 1)
    if (abs(sb_buy[year].x) > 1e-6):
        sb_plan.loc["Buy", year] = np.round(sb_buy[year].x, 1)
    if (abs(sb_sell[year].x) > 1e-6):
        sb_plan.loc["Sell", year] = np.round(sb_sell[year].x, 1)
sb_plan

,1,2,3,4,5
Grow,91.1,94.0,97.7,114.6,131.3
Buy,0.0,0.0,0.0,0.0,0.0
Sell,22.8,27.4,24.6,42.1,66.6


### 小母牛计划

该计划显示了在规划期内每个时期要出售和饲养的小母牛数量。例如,我们将在第2年出售40.8头小母牛并饲养11.6头小母牛。

In [19]:
rows = ["Sell", "Raise"]
columns = years.copy()
livestock_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)

for year in years:
    if (abs(hf_sell[year].x) > 1e-6):
        livestock_plan.loc["Sell", year] = np.round(hf_sell[year].x, 1)
    if (abs(newborn[year].x) > 1e-6):
        livestock_plan.loc["Raise", year] = np.round(newborn[year].x, 1)
livestock_plan

,1,2,3,4,5
Sell,30.9,40.8,57.4,57.0,50.9
Raise,22.8,11.6,0.0,0.0,0.0


---
## 参考文献

H. Paul Williams, 《数学规划中的模型构建》，第五版。

版权所有 © 2020 Gurobi Optimization, LLC